In [1]:
from kafka import KafkaConsumer
import json
import matplotlib.pyplot as plt
import time

In [2]:
# Initialize data storage for plotting
timestamps = []
water_temperatures = []
ph_levels = []
turbidities = []
dissolved_oxygen_levels = []

In [3]:
# Kafka configuration
def initialize_consumer():
    kafka_topic = "water_quality"
    kafka_bootstrap_servers = ["localhost:9092"]

    # Create Kafka consumer
    consumer = KafkaConsumer(
        kafka_topic,
        bootstrap_servers=kafka_bootstrap_servers,
        value_deserializer=lambda m: json.loads(m.decode('utf-8')),
        auto_offset_reset='latest',
        enable_auto_commit=True
    )
    return consumer

In [4]:
# Receive all published messages and update plot
def update_plot(consumer):
    try:
        for message in consumer:
            # Parse the message
            sensor_data = message.value
            print(f"Received: {sensor_data}")

            # Update data storage
            timestamps.append(sensor_data['timestamp'])
            water_temperatures.append(sensor_data['water_temperature'])
            ph_levels.append(sensor_data['ph_level'])
            turbidities.append(sensor_data['turbidity'])
            dissolved_oxygen_levels.append(sensor_data['dissolved_oxygen'])

            # Keep only the last 100 entries for plotting
            if len(timestamps) > 100:
                timestamps.pop(0)
                water_temperatures.pop(0)
                ph_levels.pop(0)
                turbidities.pop(0)
                dissolved_oxygen_levels.pop(0)

            # Clear the current axes and redraw the plots
            plt.figure(figsize=(10, 8))

            plt.subplot(2, 2, 1)
            plt.plot(timestamps, water_temperatures, label="Water Temperature", color="blue")
            plt.title("Water Temperature")
            plt.ylabel("°C")

            plt.subplot(2, 2, 2)
            plt.plot(timestamps, ph_levels, label="pH Level", color="green")
            plt.title("pH Level")
            plt.ylabel("pH")

            plt.subplot(2, 2, 3)
            plt.plot(timestamps, turbidities, label="Turbidity", color="orange")
            plt.title("Turbidity")
            plt.ylabel("NTU")

            plt.subplot(2, 2, 4)
            plt.plot(timestamps, dissolved_oxygen_levels, label="Dissolved Oxygen", color="red")
            plt.title("Dissolved Oxygen")
            plt.ylabel("mg/L")

            plt.tight_layout()

            # Save the plot as an image
            plt.savefig(f"water_quality_plot.png")
            plt.close()

            break  # Process one message at a time
    except KeyboardInterrupt:
        print("Stopped consuming messages.")
        consumer.close()

In [5]:
consumer = initialize_consumer()
print("Subscribed to Kafka topic 'water_quality'.")

try:
    while True:
        update_plot(consumer)
except KeyboardInterrupt:
    print("Stopped visualization.")
    consumer.close()

Subscribed to Kafka topic 'water_quality'.
Received: {'timestamp': 1741103565, 'water_temperature': 28.507685878253902, 'ph_level': 8.2762785755026, 'turbidity': 28.62, 'dissolved_oxygen': 5.23}
Received: {'timestamp': 1741103566, 'water_temperature': 27.678568993169804, 'ph_level': 7.9936444207705, 'turbidity': 44.47, 'dissolved_oxygen': 6.9}
Received: {'timestamp': 1741103567, 'water_temperature': 27.92850155776839, 'ph_level': 7.6646530827922295, 'turbidity': 11.11, 'dissolved_oxygen': 10.27}
Received: {'timestamp': 1741103568, 'water_temperature': 29.154910779588246, 'ph_level': 8.485690482621306, 'turbidity': 43.75, 'dissolved_oxygen': 8.24}
Received: {'timestamp': 1741103569, 'water_temperature': 28.966544336488635, 'ph_level': 7.6750608849416855, 'turbidity': 9.59, 'dissolved_oxygen': 8.96}
Received: {'timestamp': 1741103570, 'water_temperature': 28.804344796861216, 'ph_level': 8.294295689740332, 'turbidity': 45.75, 'dissolved_oxygen': 11.65}
Received: {'timestamp': 1741103571, 